# Create tables in AWS DynamoDb


In [ ]:
import json
import requests
import csv
import io
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
FILEPATH = "../data"

class PubChemBioAssayRecordFromCsv(object):
    def __init__(self, assay_id):
        self.assay_id = assay_id
        result = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{0}/CSV".format(self.assay_id))
        self.record = result.text
        
    def save_csv(self):
        with open (os.path.join(FILEPATH, "pubchem_aid{0}.csv".format(self.assay_id)),"w") as f:
            f.write(self.record)
    
    def csv2df(self):
        result = io. StringIO(self.record)
        df = pd.read_csv(result, sep=",")
        st_cols = []
        exp_cols = []
        for c in list(df.columns):
            if "PUBCHEM" in c:
                st_cols += [c]
            else:
                exp_cols += [c]
                df["{}_TYPE".format(c)] = np.nan*(df.shape[0])
                df["{}_DESCR".format(c)] = np.nan*(df.shape[0])
                df["{}_UNIT".format(c)] = np.nan*(df.shape[0])
        return df, st_cols, exp_cols
    
    def get_smiles_from_cid(self):
        df = self.csv2df

In [ ]:
r = PubChemBioAssayRecordFromCsv(400)

In [ ]:
df, st_cols, exp_cols = r.csv2df()

In [ ]:
exp_cols

In [ ]:
cols_to_order = []
for c in df.columns:
    if "PUBCHEM" not in c:
        cols_to_order += [c]

In [ ]:
new_order = []

for c in exp_cols:
    for c2 in cols_to_order:
        if c in c2:
            new_order += [c2]
all_cols = st_cols + new_order
all_cols

df = df[all_cols]

for c in exp_cols:
    df[0][c]

<xs:enumeration value="ppt" ncbi:intvalue="1"/>
<xs:enumeration value="ppm" ncbi:intvalue="2"/>
<xs:enumeration value="ppb" ncbi:intvalue="3"/>
<xs:enumeration value="mm" ncbi:intvalue="4"/>
<xs:enumeration value="um" ncbi:intvalue="5"/>
<xs:enumeration value="nm" ncbi:intvalue="6"/>
<xs:enumeration value="pm" ncbi:intvalue="7"/>
<xs:enumeration value="fm" ncbi:intvalue="8"/>
<xs:enumeration value="mgml" ncbi:intvalue="9"/>
<xs:enumeration value="ugml" ncbi:intvalue="10"/>
<xs:enumeration value="ngml" ncbi:intvalue="11"/>
<xs:enumeration value="pgml" ncbi:intvalue="12"/>
<xs:enumeration value="fgml" ncbi:intvalue="13"/>
<xs:enumeration value="m" ncbi:intvalue="14"/>
<xs:enumeration value="percent" ncbi:intvalue="15"/>
<xs:enumeration value="ratio" ncbi:intvalue="16"/>
<xs:enumeration value="sec" ncbi:intvalue="17"/>
<xs:enumeration value="rsec" ncbi:intvalue="18"/>
<xs:enumeration value="min" ncbi:intvalue="19"/>
<xs:enumeration value="rmin" ncbi:intvalue="20"/>
<xs:enumeration value="day" ncbi:intvalue="21"/>
<xs:enumeration value="rday" ncbi:intvalue="22"/>
<xs:enumeration value="ml-min-kg" ncbi:intvalue="23"/>
<xs:enumeration value="l-kg" ncbi:intvalue="24"/>
<xs:enumeration value="hr-ng-ml" ncbi:intvalue="25"/>
<xs:enumeration value="cm-sec" ncbi:intvalue="26"/>
<xs:enumeration value="mg-kg" ncbi:intvalue="27"/>
<xs:enumeration value="none" ncbi:intvalue="254"/>
<xs:enumeration value="unspecified" ncbi:intvalue="255"/